# Occasion Classfier using Guest Count Bins and Heuristics

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path+"/heuristics")

In [26]:
module_path

'/Users/danielastepanov/Documents/Bar-Ilan/b_semester18/DS_project/BeerW-BigData-Project'

In [3]:
from occasion_classifier import shrink_orders_to_table
from occasion_classifier import classify

In [4]:
hockey = pd.read_csv("../data/hockey_3_text_processed.csv")
silvester = pd.read_csv("../data/silvester_3_text_processed.csv")
valentine = pd.read_csv("../data/valentine_3_text_processed.csv")

data = pd.concat([hockey, silvester, valentine], axis=0)

## Using clean data (pre-processed) and bars we deemed trsutworthy, we use the Guest_Count feature to divide our orders into the following bins:

### Bins:
### - 1
### - 2
### - 3-5
### - 6+

## For each bin, we describe the chosen occasions

## Common Occasions to all bins:
* **BREAKFAST**
* **LUNCH**
* **DINNER**
* **DRINKING**
* **UNK**

### Category 1:
1. **LUNCH** - time of day must be `lunch time`, and must contain at least 1 large meal and 0 or 1 drinks.
2. **MUNCH** - time of day must be `late_night`, contains only meals that are not large
3. **DINNER** - time of day either `dinner` or `late_night`, contains at least 1 large meal, and up to 2 drinks
4. **CASUAL DRINK** - any time of day, ratio of food to drinks less than 1.5. up to 3 drinks can have <=1.2 L of beer.
5. **DRINKING** - any time of day, ratio of drinks to food larger than 1.5.
6. **NOT_1** - either too many items ordered in a single step, or "kid" in one of the titles.
7. **UNK** - undealt with

### Category 2:
1. **LUNCH** 
2. **DINNER** 
3. **DRINKING**
4. **ROMANTIC_DATE**
5. **FANCY_DATE**
6. **MALES_ONLY**
7. **JUST_EATING**
8. **BIRTHDAY**
9. **KIDS**
10. **NOT_2**
11. **UNK**

### Category 3-5:
1. **Family Event**
2. **Drinking**
3. **Breakfast**
4. **Lunch**
5. **Dinner**
6. **Social Gathering**
7. **After Work**

### Category 6:
1. 

------------------------------------------

In [6]:
labels_1 = ["LUNCH", "MUNCH", "DINNER", "DRINKING", "CASUAL_DRINK", "NOT_1", "UNK"]
labels_2 = ["LUNCH","DINNER", "DRINKING","ROMANTIC_DATE", "FANCY_DATE", "MALES_ONLY", \
            "JUST_EATING", "BIRTHDAY", "KIDS","NOT_2" ,"UNK"]
labels_35 = ["FAMILY_EVENT", "DRINKING", "BREAKFAST", "LUNCH", "DINNER", "SOCIAL_GATHERING", "AFTER_WORK"]
labels_6 = ["Family Event","Drinking","Breakfast","Lunch","Dinner","Social Gathering","After Work"]

## We annotated ~50 tables for dev, and 25 for test, for each category bin.


# DEV results

In [5]:
import csv

results = {}
total_correct = 0
total_incorrect = 0
with open("../heuristics/dev.csv") as f:
    csv_reader = csv.reader(f, delimiter=',')
    current_cat = 0
    table_results = {}
    for row in csv_reader:
        if len(row) == 1:
            current_cat = row[0]
            results[current_cat] = {}
            table_results[current_cat] = []
            results[current_cat]["correct"] = 0
            results[current_cat]["incorrect"] = 0
            continue
            
        orders = data[data.order_id == int(row[0])]
        table = shrink_orders_to_table(orders)
        pred_occasion = classify(table)
        table_results[current_cat].append((int(row[0]), row[1], pred_occasion))
        if pred_occasion == row[1]:
            results[current_cat]["correct"] += 1
            total_correct += 1
        else:
            results[current_cat]["incorrect"] += 1
            total_incorrect += 1


# Accumulative Accuracy for all guest count bins for all occasions:


In [6]:
total_correct/(total_incorrect + total_correct)*100

85.0

## Accuracy for Category 1

In [7]:
results['1']["correct"]/(results['1']["correct"] + results['1']["incorrect"])

0.9215686274509803

## Accuracy for Category 2

In [8]:
results['2']["correct"]/(results['2']["correct"] + results['2']["incorrect"])

0.2857142857142857

## Accuracy for Category 3-5

In [9]:
results['3-5']["correct"]/(results['3-5']["correct"] + results['3-5']["incorrect"])

0.85

## Accuracy for Category 6

In [10]:
results['6']["correct"]/(results['6']["correct"] + results['6']["incorrect"])

0.8571428571428571

----------------------------------------------

# TEST Results

In [11]:
import csv

results = {}
total_correct = 0
total_incorrect = 0
with open("../heuristics/test.csv") as f:
    csv_reader = csv.reader(f, delimiter=',')
    current_cat = 0
    table_results = {}
    for row in csv_reader:
        if len(row) == 1:
            current_cat = row[0]
            results[current_cat] = {}
            table_results[current_cat] = []
            results[current_cat]["correct"] = 0
            results[current_cat]["incorrect"] = 0
            continue
            
        orders = data[data.order_id == int(row[0])]
        table = shrink_orders_to_table(orders)
        pred_occasion = classify(table)
        table_results[current_cat].append((int(row[0]), row[1], pred_occasion))
        if pred_occasion == row[1]:
            results[current_cat]["correct"] += 1
            total_correct += 1
        else:
            results[current_cat]["incorrect"] += 1
            total_incorrect += 1


# Accumulative TEST Accuracy for all guest count bins for all occasions:


In [12]:
total_correct/(total_incorrect + total_correct)*100

80.23255813953489

## Accuracy for Category 1

In [13]:
results['1']["correct"]/(results['1']["correct"] + results['1']["incorrect"])

0.7083333333333334

## Accuracy for Category 2

In [14]:
results['2']["correct"]/(results['2']["correct"] + results['2']["incorrect"])

ZeroDivisionError: division by zero

## Accuracy for Category 3-5

In [15]:
results['3-5']["correct"]/(results['3-5']["correct"] + results['3-5']["incorrect"])

0.8666666666666667

## Accuracy for Category 6

In [16]:
results['6']["correct"]/(results['6']["correct"] + results['6']["incorrect"])

0.8125

## Theres quite a gap between our dev and test accuracy for most of the categories (Category 3-5 remains above 85% for both dev and test).
## This could be due to the fact that some categories are more involved than others, as well as the amount that we have labeled. In an ideal situation, a much larger labeling effort would be done.

------------------------------------

## Lets try our classifiers on each dataset

# Hockey

In [17]:
hockey_ids = list(hockey.order_id.value_counts().keys())

In [ ]:
results = {}
current_cat = 0
table_results = []
hockey_tables = shrink_orders_to_table(hockey)
for order in hockey_ids:
    table = hockey_tables[hockey_tables.order_id == order]
    pred_occasion = classify(table)
    table_results.append((order, table.guest_count.iloc[0], pred_occasion))

# Valentine

In [44]:
valentine_ids = list(valentine.order_id.value_counts().keys())

In [ ]:
results = {}
current_cat = 0
val_results = []
for order in valentine_ids:
    orders = valentine[valentine.order_id == order]
    table = shrink_orders_to_table(orders)
    pred_occasion = classify(table)
    val_results.append((order, table.guest_count.iloc[0], pred_occasion))

# Silvester

In [44]:
silvester_ids = list(silvester.order_id.value_counts().keys())

In [ ]:
results = {}
current_cat = 0
sil_results = []
for order in silvester_ids:
    orders = silvester[silvester.order_id == order]
    table = shrink_orders_to_table(orders)
    pred_occasion = classify(table)
    sil_results.append((order, table.guest_count.iloc[0], pred_occasion))